## 2024.04.23 PlayData Data Engineering

### 머신러닝 기초  

#### 모집단과 통계적 추정
- 관측을 충분히 많이 하면 모집단의 모습을 상당히 선명하게 파악할 수 있다.
>   - 그렇지만 많은 관측을 하지 않고 모집단의 모습을 추측할 수 있어야 한다.
>   - 이것을 <span style="color:green;font-weight:bold">통계적 추정</span>이라고 말한다.

#### 통계적 추청
- 모집단이 정규 분포를 이룰 때 모집단에서 추출한 표본집단 역시 정규분포를 따르며 그 평균도 정규분포를 그대로 유지합니다.
>   - 표본평균 $ \bar x $  = 평균 $ u $ , 표본 표준 편차: $ \frac{\sigma}{\sqrt n} $
>   - 표본 분산은 정규분포를 따르지 않는다. (카이제곱 분포)

### 카이제곱 분포
- 표준 모 정규 분포에서 추출한 값과 모 평균과의 편차의 제곱의 합이 가지는 분포
- 추출한 값의 개수가 n이라고 할 때 자유도가 n-1 자유도인 카이제곱 분포를 따르게 됩니다.

### t분포
- 모집단이 정규분포 한다는 지식만 가지고 모수를 추정하였다.
- 모평균을 추정하기 위해서 나온 분포
- 데이터 샘플이 많아질 수록 정규분포를 따른다.
>   - $ T = \frac{x-u * \sqrt (n-1)}{s} $

### 가설검정
**가설검정의 절차**

    1. 가설 설정(귀무가설/대립가설)
    2. 가설 유의 수준 설정
    
**t-test**
>   - 두 모집단 간의 평균의 차이가 없다는 귀무가설을 검증하는 테스트입니다.

In [247]:
#t-test example : 주유소

import pandas as pd
import numpy as np # 계산 자체에 초점
import scipy as sc # 응용에 초점) numpy 선형대수 변환 이후 과학/공학적 계산을 위한 라이브러리

import glob

csv_list = list(glob.glob('./data/*.csv'))

engine_dfs = [pd.read_csv(x, encoding='cp949') for x in csv_list]
주유소 = pd.concat([engine_dfs[0],engine_dfs[1]])
주유소.상호.isin((주유소.groupby(by=['상호'])[['번호']].count() == 365).index)

0        True
1        True
2        True
3        True
4        True
         ... 
91119    True
91120    True
91121    True
91122    True
91123    True
Name: 상호, Length: 181714, dtype: bool

In [228]:
주유소['월'] = 주유소['기간'].apply(lambda x: str(x)[4:6])

폐업 = 주유소[['월','주소']].drop_duplicates().groupby(by=['월'])[['주소']].count()

폐업['현황'] = 폐업.주소 - 폐업.주소.shift(1)

폐업.fillna(0).rename(columns={'주소':'집계', '현황':'폐업갯수'}).reset_index()

,월,집계,폐업갯수
0,01,508,0.0
1,02,508,0.0
2,03,503,-5.0
3,04,501,-2.0
4,05,501,0.0
5,06,499,-2.0
6,07,500,1.0
7,08,499,-1.0
8,09,499,0.0
9,10,499,0.0


In [230]:
# 각 지역별 휘발유 평균 가격
주유소.groupby(by=['지역'])[['휘발유']].mean().reset_index()


,지역,휘발유
0,서울 강남구,1713.093581
1,서울 강동구,1576.685264
2,서울 강북구,1453.517525
3,서울 강서구,1488.818088
4,서울 관악구,1518.128564
5,서울 광진구,1490.597054
6,서울 구로구,1525.410383
7,서울 금천구,1477.121992
8,서울 노원구,1531.221551
9,서울 도봉구,1494.171062


In [252]:
강남구 = 주유소[주유소['지역'] == '서울 강남구']
도봉구 = 주유소[주유소['지역'] == '서울 도봉구']

도봉구.head(3)

,번호,지역,상호,주소,기간,상표,셀프여부,고급휘발유,휘발유,경유,실내등유
34210,A0009647,서울 도봉구,(주)소모에너지엔테크놀러지 신중계주유소,서울 도봉구 방학로 21 (방학동),20190101,GS칼텍스,일반,0,1398,1296,1200
34211,A0009647,서울 도봉구,(주)소모에너지엔테크놀러지 신중계주유소,서울 도봉구 방학로 21 (방학동),20190102,GS칼텍스,일반,0,1398,1296,1200
34212,A0009647,서울 도봉구,(주)소모에너지엔테크놀러지 신중계주유소,서울 도봉구 방학로 21 (방학동),20190103,GS칼텍스,일반,0,1398,1296,1200


In [250]:
sc.stats.ttest_ind(a = 강남구.groupby(by=['상호'])[['휘발유']].mean().values, b = 도봉구.groupby(by=['상호'])[['휘발유']].mean().values)

TtestResult(statistic=array([4.88857177]), pvalue=array([8.39772049e-06]), df=array([58.]))

In [ ]:
sc.stats.ttest_ind(a = 강남구.groupby(by=['상호'])[['휘발유']].mean().values, b = 도봉구.groupby(by=['상호'])[['휘발유']].mean().values)

In [80]:
주유소.head(3)

,번호,지역,상호,주소,기간,상표,셀프여부,고급휘발유,휘발유,경유,실내등유
0,A0006039,서울 강남구,(주)동하힐탑셀프주유소,서울 강남구 논현로 640,20190101,SK에너지,셀프,1673,1465,1365,0
1,A0006039,서울 강남구,(주)동하힐탑셀프주유소,서울 강남구 논현로 640,20190102,SK에너지,셀프,1673,1465,1365,0
2,A0006039,서울 강남구,(주)동하힐탑셀프주유소,서울 강남구 논현로 640,20190103,SK에너지,셀프,1673,1465,1365,0


In [253]:
import sqlalchemy
from urllib import parse

user = 'david'
password = '392766'
host='172.30.240.1'
port = 3306
database = 'encore'
password = parse.quote_plus(password)

engine = sqlalchemy.create_engine(f"mysql://{user}:{password}@{host}:{port}/{database}")